
# EDA

## Задача проекта:

    Построить модель, должна выбрать принять или отклонить сделку на бирже.
    
## Цели проекта:

    1)Проверить качество данных.
    2)Сформулировать предположения и гипотезы для дальнейшего построения модели.
    3)Определиться с параметрами модели.

## <center style="background-color:Gainsboro; width:40%;">Содержание</center>
* [The train.csv file is big](#train_csv)
* [resp](#resp)
* [weight](#weight)
* [Cumulative return](#return)
* [Time](#time)
* [The features](#features)
* [The `features.csv` file](#features_file)
* [Action](#action)
* [Missing values](#missing_values)
* [Correlation](#Pearson)

In [ ]:
# numpy
import numpy as np

# pandas stuff
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# plotting stuff
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
colorMap = sns.light_palette("blue", as_cmap=True)
#plt.rcParams.update({'font.size': 12})

# system
import warnings
warnings.filterwarnings('ignore')  
# garbage collector to keep RAM in check
import gc

<a class="anchor" id="train_csv"></a>
## <center style="background-color:Gainsboro; width:40%;">The train.csv file is big</center>

Файл train.csv весит 5.77 Гб.

In [ ]:
%%time

train_data = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [ ]:
train_data = train_data.astype({c: np.float32 for c in train_data.select_dtypes(include='float64').columns}) #limit memory use

In [ ]:
train_data.info()

In [ ]:
train_data.head()



<a class="anchor" id="return"></a>
## <center style="background-color:Gainsboro; width:40%;">resp</center>

В файле `train.csv` содержатся данные за 500 дней (т.е. данные о торговле за два года). Давайте посмотрим на совокупные значения `resp` с течением времени.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['resp']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_ylabel ("Cumulative resp", fontsize=18);
balance.plot(lw=3);
del balance
gc.collect();

Так же у нас есть 4 временных горизонта. 

    Временной горизонт - это период времени, в течение которого человек ожидает удерживать вложение до тех пор, пока ему не понадобятся деньги обратно.
    
> "*Чем длиннее временной горизонт, тем более агрессивный или рискованный портфель может построить инвестор. Чем короче временной горизонт, тем более консервативным или менее рискованным может быть инвестор.*"

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['resp']).cumsum()
resp_1= pd.Series(train_data['resp_1']).cumsum()
resp_2= pd.Series(train_data['resp_2']).cumsum()
resp_3= pd.Series(train_data['resp_3']).cumsum()
resp_4= pd.Series(train_data['resp_4']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
balance.plot(lw=3)
resp_1.plot(lw=3)
resp_2.plot(lw=3)
resp_3.plot(lw=3)
resp_4.plot(lw=3)
plt.legend(loc="upper left");
del resp_1
del resp_2
del resp_3
del resp_4
gc.collect();

Мы видим что `resp` (синяя) наиболее точно соответствует временному горизонту 4 (`resp_4` - самая верхняя кривая, выделенная фиолетовым цветом).


<a class="anchor" id="weight"></a>
## <center style="background-color:Gainsboro; width:40%;">weight</center>

> * Каждой сделке соответствуют значения `weight` и `resp`, которые вместе представляют прибыль от сделки.
Сделки с `weight = 0` были намеренно включены в набор данных для полноты, хотя такие сделки не учитываются при выставлении баллов *.

In [ ]:
percent_zeros = (100/train_data.shape[0])*((train_data.weight.values == 0).sum())
print('Percentage of zero weights is: %i' % percent_zeros +"%")

Посмотрим, есть ли отрицательные веса. Отрицательный вес был бы бессмысленным, но мало ли ...

In [ ]:
min_weight = train_data['weight'].min()
print('The minimum weight is: %.2f' % min_weight)

Теперь найдем максимальный вес

In [ ]:
max_weight = train_data['weight'].max()
print('The maximum weight was: %.2f' % max_weight)

Посмотрим в какой день

In [ ]:
train_data[train_data['weight']==train_data['weight'].max()]

Посмотрим на гистограмму ненулевых весов

In [ ]:
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['weight'], 
             bins=1400, 
             kde_kws={"clip":(0.001,1.4)}, 
             hist_kws={"range":(0.001,1.4)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of non-zero weights", size=14)
plt.show();
del values
gc.collect();

Похоже, что есть два пика, один с `weight` ≈ 0,17 и более низкий, более широкий пик с `weight` ≈ 0,34. Может ли это указывать на два основных распределения, которые мы видим здесь, наложенные друг на друга? Может быть, одно распределение весов соответствует продаже, а другое - покупке?

Построим график с логарифмом весов.

In [ ]:
train_data_nonZero = train_data.query('weight > 0').reset_index(drop = True)
plt.figure(figsize = (10,4))
ax = sns.distplot(np.log(train_data_nonZero['weight']), 
             bins=1000, 
             kde_kws={"clip":(-4,5)}, 
             hist_kws={"range":(-4,5)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the logarithm of the non-zero weights", size=14)
plt.show();
gc.collect();

Распределение уже более нормальное, но кажется, будто их 2 наложенных друг на друга.


<a class="anchor" id="return"></a>
## <center style="background-color:Gainsboro; width:40%;">Cumulative return</center>


Давайте посмотрим на совокупную дневную доходность с течением времени, которая определяется как `weight`, умноженный на соответствующий `resp`.

In [ ]:
train_data['weight_resp']   = train_data['weight']*train_data['resp']
train_data['weight_resp_1'] = train_data['weight']*train_data['resp_1']
train_data['weight_resp_2'] = train_data['weight']*train_data['resp_2']
train_data['weight_resp_3'] = train_data['weight']*train_data['resp_3']
train_data['weight_resp_4'] = train_data['weight']*train_data['resp_4']

fig, ax = plt.subplots(figsize=(15, 5))
resp    = pd.Series(1+(train_data.groupby('date')['weight_resp'].mean())).cumprod()
resp_1  = pd.Series(1+(train_data.groupby('date')['weight_resp_1'].mean())).cumprod()
resp_2  = pd.Series(1+(train_data.groupby('date')['weight_resp_2'].mean())).cumprod()
resp_3  = pd.Series(1+(train_data.groupby('date')['weight_resp_3'].mean())).cumprod()
resp_4  = pd.Series(1+(train_data.groupby('date')['weight_resp_4'].mean())).cumprod()
ax.set_xlabel ("Day", fontsize=18)
ax.set_title ("Cumulative daily return for resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
resp.plot(lw=3, label='resp x weight')
resp_1.plot(lw=3, label='resp_1 x weight')
resp_2.plot(lw=3, label='resp_2 x weight')
resp_3.plot(lw=3, label='resp_3 x weight')
resp_4.plot(lw=3, label='resp_4 x weight')
# day 85 marker
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
plt.legend(loc="lower left");

Мы видим, что самые короткие временные горизонты `resp_1`, `resp_2` и `resp_3`, представляющие более консервативную стратегию, дают самый низкий доход.

Теперь мы построим гистограмму совокупного дохода (после удаления нулевых весов).

In [ ]:
train_data_no_0 = train_data.query('weight > 0').reset_index(drop = True)
train_data_no_0['wAbsResp'] = train_data_no_0['weight'] * (train_data_no_0['resp'])
#plot
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data_no_0['wAbsResp'], 
             bins=1500, 
             kde_kws={"clip":(-0.02,0.02)}, 
             hist_kws={"range":(-0.02,0.02)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the weights * resp", size=14)
plt.show();

<a class="anchor" id="time"></a>
## <center style="background-color:Gainsboro; width:40%;">Time</center>


Построим график количества ts_id в день. 

    Примечание:  начали рисовать вертикальную пунктирную линию на графиках, потому что появилось подозрение, что примерно в это время на рынке произошло изменение (возможно, это означает возврат рынка к импульсному рынку или наоборот).

In [ ]:
trades_per_day = train_data.groupby(['date'])['ts_id'].count()
fig, ax = plt.subplots(figsize=(15, 5))
plt.plot(trades_per_day)
ax.set_xlabel ("Day", fontsize=18)
ax.set_title ("Total number of ts_id for each day", fontsize=18)
# day 85 marker
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
plt.show()

Если предположить, что торговый день длится 6,5 часов (т.е. 23400 секунд), то посмотрим среднее время между сделками за каждый день.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
plt.plot(23400/trades_per_day)
ax.set_xlabel ("Day", fontsize=18)
ax.set_ylabel ("Av. time between trades (s)", fontsize=18)
ax.set_title ("Average time between trades for each day", fontsize=18)
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
ax.set_ylim(ymin=0)
ax.set_ylim(ymax=12)
plt.show()

<!-- Here is a histogram of the number of trades per day (it has been [suggested](https://www.kaggle.com/c/jane-street-market-prediction/discussion/201930#1125847) that the number of trades per day is an indication of the [volatility](https://www.investopedia.com/terms/v/volatility.asp) that day) -->

Вот гистограмма количества сделок в день (было высказано предположение, что количество сделок в день является показателем волатильности в тот день).

Волатильность - это статистическая мера разброса доходности для ценной бумаги или рыночного индекса. В большинстве случаев, чем выше волатильность, тем выше риск. 

In [ ]:
plt.figure(figsize = (12,4))
# the minimum has been set to 1000 so as not to draw the partial days like day 2 and day 294
# the maximum number of trades per day is 18884
# I have used 125 bins for the 500 days
ax = sns.distplot(trades_per_day, 
             bins=125, 
             kde_kws={"clip":(1000,20000)}, 
             hist_kws={"range":(1000,20000)},
             color='darkcyan', 
             kde=True);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Number of trades per day", size=14)
plt.show();

Если это так, то «волатильные» дни, скажем, с более чем 9 тыс. Сделок (т.е. `ts_id`) в день, следующие:

In [ ]:
volitile_days = pd.DataFrame(trades_per_day[trades_per_day > 9000])
volitile_days.T

Интересно отметить, что почти все дни с большим объемом торгов приходятся на дни до 85 дня и на сам этот день.


<a class="anchor" id="features"></a>
## <center style="background-color:Gainsboro; width:40%;">The features</center>

> "*Этот набор данных содержит анонимный набор функций, feature_ {0 ... 129}, представляющих реальные данные фондового рынка*"

### feature_0

Прежде всего, `feature_0` кажется немного необычным, поскольку он состоит исключительно из целых чисел` + 1` или `-1`:

In [ ]:
train_data['feature_0'].value_counts()

Кроме того, `feature_0` - единственная функция в файле `features.csv`, у которой нет тегов True.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
feature_0 = pd.Series(train_data['feature_0']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_ylabel ("feature_0 (cumulative)", fontsize=18);
feature_0.plot(lw=3);

Возможно, сделка, инициированная покупкой, помечается как «1», а сделка, инициированная продажей, - как «-1».



### feature_{1...129}
Похоже, есть четыре основных «типа» функций, вот график примера каждой из них:

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,figsize=(20,10))

ax1.plot((pd.Series(train_data['feature_1']).cumsum()), lw=3, color='red')
ax1.set_title ("Linear", fontsize=22);
ax1.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax1.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax1.set_xlim(xmin=0)
ax1.set_ylabel ("feature_1", fontsize=18);

ax2.plot((pd.Series(train_data['feature_3']).cumsum()), lw=3, color='green')
ax2.set_title ("Noisy", fontsize=22);
ax2.axvline(x=514052, linestyle='--', alpha=0.3, c='red', lw=2)
ax2.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax2.set_xlim(xmin=0)
ax2.set_ylabel ("feature_3", fontsize=18);

ax3.plot((pd.Series(train_data['feature_55']).cumsum()), lw=3, color='darkorange')
ax3.set_title ("Hybryd (Tag 21)", fontsize=22);
ax3.set_xlabel ("Trade", fontsize=18)
ax3.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax3.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax3.set_xlim(xmin=0)
ax3.set_ylabel ("feature_55", fontsize=18);

ax4.plot((pd.Series(train_data['feature_73']).cumsum()), lw=3, color='blue')
ax4.set_title ("Negative", fontsize=22)
ax4.set_xlabel ("Trade", fontsize=18)
ax4.set_ylabel ("feature_73", fontsize=18);
gc.collect();

### 'Linear' features
* 1 
* 7, 9, 11, 13, 15
* 17, 19, 21, 23, 25
* 18,  20,  22,  24, 26
* 27, 29, 21, 33, 35
* 28, 30, 32, 34, 36
* 84, 85, 86, 87, 88
* 90, 91, 92, 93, 94
* 96, 97, 98, 99, 100
* 102 , 103, 104, 105, 106

а также
41, 46, 47, 48, 49, 50, 51, 53, 54, 69, 89, 95 , 101, 107, 108, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 122,124.



### 'Noisy' features
* 3, 4, 5, 6
* 8, 10, 12, 14, 16
* 37, 38, 39, 40
* 72, 73, 74, 75, 76
* 78, 79, 80, 81, 82
* 83



### 'Negative' features
Features 73, 75, 76, 77 (noisy), 79, 81(noisy), 82. Все они находятся в **Tag 23**.

### 'Hybrid' features (Tag 21): 
55, 56, 57, 58, 59.

Они начинаются шумно, с заметными почти прерывистыми ступенями вокруг 0.2M, 0.5M, and 0.8M trade marks, затем идут линейно. Эти пять функций составляют набор «** Tag 21 **»:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
feature_55= pd.Series(train_data['feature_55']).cumsum()
feature_56= pd.Series(train_data['feature_56']).cumsum()
feature_57= pd.Series(train_data['feature_57']).cumsum()
feature_58= pd.Series(train_data['feature_58']).cumsum()
feature_59= pd.Series(train_data['feature_59']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative plot for the 'Tag 21' features (55-59)", fontsize=18)
ax.axvline(x=514052, linestyle='--', alpha=0.3, c='black', lw=1)
ax.axvspan(0,  514052, color=sns.xkcd_rgb['grey'], alpha=0.1)
feature_55.plot(lw=3)
feature_56.plot(lw=3)
feature_57.plot(lw=3)
feature_58.plot(lw=3)
feature_59.plot(lw=3)
plt.legend(loc="upper left");
gc.collect();


<a class="anchor" id="features_file"></a>
## <center style="background-color:Gainsboro; width:40%;">The features.csv file</center>

Нам также предоставляется файл `features.csv`, который содержит «метаданные, относящиеся к анонимным функциям». Давайте взглянем на него, где `1` - `True`, а `0` - `False`. В файле есть 29 «тегов», связанных с каждой функцией.

In [ ]:
feature_tags = pd.read_csv("../input/jane-street-market-prediction/features.csv" ,index_col=0)
# конвертируем в бинарный формат
feature_tags = feature_tags*1
# строим транспонированный dataframe
feature_tags.T.style.background_gradient(cmap='Oranges')

Суммируем количество тегов для каждой функции:

In [ ]:
tag_sum = pd.DataFrame(feature_tags.T.sum(axis=0),columns=['Number of tags'])
tag_sum.T

Мы видим, что у всех функций есть хотя бы один тег, а у некоторых - четыре. Все, кроме функции feature_0, у которой вообще нет тегов.



<a class="anchor" id="action"></a>
## <center style="background-color:Gainsboro; width:40%;">Action</center>

Задачей модели является действие: 1 - совершить сделку и 0 - передать ее. В связи с этим добавим новый столбец в наш тестовый набор данных под названием action, так что если `resp` положительный, тогда `action=1`, иначе `action=0`, т.е.

In [ ]:
train_data['action'] = ((train_data['resp'])>0)*1

Давайте теперь сравним общее действие с бездействием

In [ ]:
train_data['action'].value_counts()

Мы видим, что, в целом, сделки совершаются чуть чаще.

Как это выглядит ежедневно?

In [ ]:
daily_action_sum   = train_data['action'].groupby(train_data['date']).sum()
daily_action_count = train_data['action'].groupby(train_data['date']).count()
daily_ratio        = daily_action_sum/daily_action_count
# now plot
fig, ax = plt.subplots(figsize=(15, 5))
plt.plot(daily_ratio)
ax.set_xlabel ("Day", fontsize=18)
ax.set_ylabel ("ratio", fontsize=18)
ax.set_title ("Daily ratio of action to inaction", fontsize=18)
plt.axhline(0.5, linestyle='--', alpha=0.85, c='r');
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
plt.show();

Ежедневные действия довольно последовательны; нет явных еженедельных / ежемесячных / сезонных изменений и т. д.

In [ ]:
daily_ratio_mean = daily_ratio.mean()
print('The mean daily ratio is %.3f' % daily_ratio_mean)

In [ ]:
daily_ratio_max = daily_ratio.max()
print('The maximum daily ratio is %.3f' % daily_ratio_max)

<a class="anchor" id="missing_values"></a>
## <center style="background-color:Gainsboro; width:60%;">Missing values</center>


In [ ]:
#missing_data = pd.DataFrame(train_data.isna().sum().sort_values(ascending=False),columns=['Total missing'])
#missing_data.T

gone = train_data.isnull().sum()
px.bar(gone, color=gone.values, title="Total number of missing values for each column").show()

Прежде всего,

79,6% всех недостающих данных находится в группе **Tag 4**, которая представляет features `resp_1`

15,2% отсутствующих данных находятся в группе **Tag 3**, которая представляет features `resp_2`

В целом features, связанные с `resp_1` и `resp_2`, составляют> 95% всех недостающих данных.



<a class="anchor" id="Pearson"></a>
## <center style="background-color:Gainsboro; width:90%;"> Correlation </center>

Взглянем на данные, используя матрицу корреляции Пирсона (это **большая** матрица!), где красный цвет указывает на положительную линейную корреляцию, а синий указывает на линейную отрицательную корреляцию:

In [ ]:
train_data.corr(method='pearson').style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)


### High correlations
Теперь мы найдем пары признаков с корреляцией > |0.9|:

In [ ]:
features_train_data  = train_data.iloc[:,7:137]

In [ ]:
def corrFilter(x: pd.DataFrame, bound: float):
    xCorr = x.corr()
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    return xFlattened

high_correlations=corrFilter(features_train_data, .9).to_frame()

In [ ]:
high_correlations

Удалим парные признаки с корреляцией > |0.9|

Дальнейшая обработка данных и подбор моделей в следующем ноутбуке.